# 准备数据

In [1]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string
from alignment_utils import align_answers_in_questions

In [2]:
import os
import glob
from pathlib import Path

examination_paper_list = []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": str(file),
            "text": one_file_per_process(f.read())
        })
len(examination_paper_list)

1615

In [3]:
import re

def type_of_judgment(text):
    # 拆分成行
    lines = text.splitlines()
    question_number_list = []
    
    pattern = r'^[一二三四五六七八九十][、．.]'
    for i in range(len(lines)):
        line = lines[i]
        match = re.match(pattern, line)
        if match:
#             print(match.group(0)[0])
            if i+1 < len(lines):
                next_line = lines[i+1]
                question_number_list.append(extract_and_combine_numbers(next_line))
    
    # 计算1在数组中出现的次数
    count_of_ones = question_number_list.count(1)
    
    return count_of_ones > len(question_number_list) / 2

# 寻找题目

In [4]:
def get_all_question(text):
    # 拆分成行
    lines = text.splitlines()
    
    # 定义不准确的题目列表    
    inaccuracy_question = []
    
    # 从0的位置寻找第一道题
    index = find_next_question_index(0, lines)
    
    while index < len(lines):
        # 寻找下一个题目的index
        next_index = find_next_question_index(index, lines)
        
        inaccuracy_question.append("\n".join(lines[index: next_index]))
        index = next_index
#     for i in inaccuracy_question:
#         print(i)
#         print()
#     print( [extract_and_combine_numbers(topic) for topic in inaccuracy_question])
    # 通过"最长递增子序列"寻找每个精准的题目所在inaccuracy_question对应的下标
    all_question_indexs = longest_increasing_subsequence_index(inaccuracy_question)
    
    # 定义准确的题目列表    
    all_question = []
    # index为all_question_indexs的下标，all_question_indexs[index]为inaccuracy_question的下标
    for index, question_index in enumerate(all_question_indexs):
        if index == len(all_question_indexs) -1 :
            all_question.append("\n".join(inaccuracy_question[question_index:]))
            break
            
        all_question.append("\n".join(inaccuracy_question[question_index:all_question_indexs[index+1]]))
    
    if not all_question:
        return None, None
    
    
    # 尝试寻找用于分割答题区与答案区的字符串，返回值为int/str，如果是str则是分割的字符串
    # 本质是在"all_question[-1]"寻找答案关键字等字样
    answer_split_str = find_answer_split_str(all_question)
    
    if isinstance(answer_split_str, str):
        # 如果找到这个拆分的字符串了，则先把最后一道题的内容进行拆分
        all_question[-1] = all_question[-1].split(answer_split_str)[0]
        
    if text.splitlines()[0] in all_question[-1]:
        answer_split_str = text.splitlines()[0]
        # 看看试卷的title是否出现在"all_question[-1]"位置，如果出现则删除
        all_question[-1] = all_question[-1].replace("\n"+text.splitlines()[0]+"\n","")

    return all_question, answer_split_str

# 创建测试样本

In [10]:
#994 题目为 图片+文字（无法用规则提取）
#1087 一 1.2.3 二 1.2.3 （规则还没写）
#1482 解析错误 暂未解决（规则BUG）

In [30]:
import random
random_number = random.randint(0, len(examination_paper_list))
print(random_number)
test_text = examination_paper_list[random_number]["text"]
# test_text = examination_paper_list[83]["text"]

1059


In [31]:
test_text.splitlines()

['北师大版小学五年级上册数学第3单元《倍数与因数》单元检测1（附答案）',
 '1\\. 填一填。',
 '(1)在0.7、8、1、－3、97、0、中，整数有( )，自然数有( )。',
 '(2)最小的奇数是( )，最小的质数是( )，最小的合数是( )。',
 '(3)50以内8的倍数有( )。',
 '(4)28的因数中，最小的是( )，最大的是( )。',
 '(5)20以内既是合数又是奇数的数有( )；既是质数又是偶数的数有( )。',
 '2．辨一辨。(对的在括号里打"√"，错的打"×")',
 '(1)合数的因数至少有3个。 来源：www.bcjy123.com/tiku/ ( )',
 '(2)在1，2，3，4，5......中，除了质数以外都是合数。 ( )',
 '(3)12的因数一定小于12。 ( )',
 '(4)所有的奇数都是质数，所有的偶数都是合数。 ( )',
 '(5)6的所有倍数都是合数。 ( )',
 '3．选一选。(把正确答案的序号填在括号里)',
 '(1)，b是两个不为0的整数，8＝b，b是的( )。',
 '①质数 ②因数 ③倍数',
 '(2)用1、4、7三个数字组成的三位数，都是( )的倍数。',
 '①2 ②3 ③7',
 '(3)20以内所有质数的和是( )。',
 '①75 ②77 ③86',
 '(4)两个质数相乘，积一定是( )。',
 '①质数 ②合数 ③奇数',
 '(5)个位上是0的自然数，一定( )。',
 '①是2和5的倍数 ②不是3的倍数 ③是2和3的倍数',
 '4．在□里填上合适的数字。',
 '7□ 2□ 1□□ □4',
 '质数。 合数。 2、3、5的倍数。 3的倍数。',
 '5□ □3 9□ 1□',
 '5的倍数， 3的倍数， 最大的两 有因数1，2，',
 '奇数。 合数。 位数，奇数。 3，4，6，12。',
 '5\\. 把下面的数填在合适的圈里。来源：www.bcjy123.com/tiku/',
 '45 28 1996 2007 180 30 95 43',
 '2的倍数 3的倍数 5的倍数 2、3、5的倍数',
 '6．做一做。 来源：www.bcjy123.com/tiku/',
 '(1)分一分：在括号里填上合适的质数。',
 '6＝( )＋( ) 30＝( )

In [32]:
# split_str用于进行下一步的分析（有可能这份试卷是有答题区和答案区，也有可能每道题的答案都在题的下方，还有可能根本没有答案）
all_question, split_str = get_all_question(test_text)
print(f"split_str:\n{split_str}")
if split_str in [-1, 0]:
    print("此试卷无答案")
all_question

split_str:
参考答案


['(1)在0.7、8、1、－3、97、0、中，整数有( )，自然数有( )。',
 '(2)最小的奇数是( )，最小的质数是( )，最小的合数是( )。',
 '(3)50以内8的倍数有( )。\n(4)28的因数中，最小的是( )，最大的是( )。\n(5)20以内既是合数又是奇数的数有( )；既是质数又是偶数的数有( )。\n2．辨一辨。(对的在括号里打"√"，错的打"×")\n(1)合数的因数至少有3个。 来源：www.bcjy123.com/tiku/ ( )\n(2)在1，2，3，4，5......中，除了质数以外都是合数。 ( )\n(3)12的因数一定小于12。 ( )\n(4)所有的奇数都是质数，所有的偶数都是合数。 ( )\n(5)6的所有倍数都是合数。 ( )\n3．选一选。(把正确答案的序号填在括号里)\n(1)，b是两个不为0的整数，8＝b，b是的( )。\n①质数 ②因数 ③倍数\n(2)用1、4、7三个数字组成的三位数，都是( )的倍数。\n①2 ②3 ③7\n(3)20以内所有质数的和是( )。\n①75 ②77 ③86\n(4)两个质数相乘，积一定是( )。\n①质数 ②合数 ③奇数\n(5)个位上是0的自然数，一定( )。\n①是2和5的倍数 ②不是3的倍数 ③是2和3的倍数',
 '4．在□里填上合适的数字。\n7□ 2□ 1□□ □4\n质数。 合数。 2、3、5的倍数。 3的倍数。\n5□ □3 9□ 1□\n5的倍数， 3的倍数， 最大的两 有因数1，2，\n奇数。 合数。 位数，奇数。 3，4，6，12。',
 '5\\. 把下面的数填在合适的圈里。来源：www.bcjy123.com/tiku/\n45 28 1996 2007 180 30 95 43\n2的倍数 3的倍数 5的倍数 2、3、5的倍数',
 '6．做一做。 来源：www.bcjy123.com/tiku/\n(1)分一分：在括号里填上合适的质数。\n6＝( )＋( ) 30＝( )＋( )\n24＝( )＋( ) 28＝( )＋( )\n(2)圈一圈：在下表中先用"○"圈出2的倍数，再用"△"圈出3的倍数。\n  ---- ---- ---- ---- ---- ---- ---- ---- ---- ----\n  1    2    3    4

In [33]:
if split_str == 1:
    for row in align_answers_in_questions(all_question):
        print(row)
#         print(row["question"])
#         print("-------------------------------------------------------")
#         print(f"{row['answer']}")
#         print()
        print("============================================================")

# 寻找答案（试卷存在答题区与答案区）

In [34]:
def get_all_answer_sequence(answer_area_string):
    lines = answer_area_string.splitlines()
    
    inaccuracy_answers = []

    index = find_next_question_index(0, lines)
    while index < len(lines):
#         print(lines[index])
#         print()
        next_index = find_next_question_index(index, lines)
       
        inaccuracy_answers.append("\n".join(lines[index: next_index]))
        index = next_index
#     print("============")
#     for i in inaccuracy_answers:
#         print(i)
#         print()
        
#     print([extract_and_combine_numbers(topic) for topic in inaccuracy_answers])
    inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)
   
    processed_inaccuracy_answers = []
    for index, answer_index in enumerate(inaccuracy_answer_indexes):
        if index == len(inaccuracy_answer_indexes) -1 :
            processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
            break
        processed_inaccuracy_answers.append("\n".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))
        
    return refine_answers(processed_inaccuracy_answers)[::-1]

In [35]:
answer_str = generate_answer_area_string(test_text, split_str)

In [36]:
answer_str.splitlines()

['',
 '1．(1)8 1 －3 97 0；8 1 97 0',
 '(2)1 2 4',
 '(3)8 16 24 32 40 48',
 '(4)1 28',
 '(5)9、15 2',
 '2．(1)√ (2)× (3)× (4)× (5)√',
 '3．(1)③ (2)② (3)② (4)② (5)①',
 '4．71 73 79；20 21 22 24 25 26 27 28；120 150 180；24 54 84；55；33 63 93；99；12',
 '5．2的倍数：28 1996 180 30；',
 '3的倍数：45 2007 180 30；',
 '5的倍数：45 180 30 95；',
 '2、3、5的倍数：180 30。',
 '6．(1)3＋3 11＋13或7＋17 7＋23或13＋17或11＋19 5＋23或11＋17',
 '(2)圈数略 6 (3)略',
 '7．42人',
 '8．可能是2人、3人、4人、6人、8人、12人、24人。',
 '9．(1)小鸭渡河的次数是偶数。因为游一个"来回"是渡河两次，是个偶数，游若干个"来回"又回到右岸就是若干个偶数相加，所以总的渡河次数是偶数。',
 '(2)到左岸。因为渡河1次、3次...等奇数次以后肯定到达左岸。',
 '来源：www.bcjy123.com/tiku/',
 '来源：www.bcjy123.com/tiku/']

In [37]:
for i in get_all_answer_sequence(answer_area_str_process(answer_str)):
    print(i)
    print()

1．(1)8 1 －3 97 0；8 1 97 0

(2)1 2 4

(3)8 16 24 32 40 48

(4)1 28

(5)9、15 2
2．(1)√ (2)× (3)× (4)× (5)√
3．(1)③ (2)② (3)② (4)② (5)①
4．71 73 79；20 21 22 24 25 26 27 28；120 150 180；24 54 84；55；33 63 93；99；12
5．2的倍数：28 1996 180 30；
3的倍数：45 2007 180 30；
5的倍数：45 180 30 95；
2、3、5的倍数：180 30。

6．(1)3＋3 11＋13或7＋17 7＋23或13＋17或11＋19 5＋23或11＋17
(2)圈数略 6 (3)略

7．42人

8．可能是2人、3人、4人、6人、8人、12人、24人。

9．(1)小鸭渡河的次数是偶数。因为游一个"来回"是渡河两次，是个偶数，游若干个"来回"又回到右岸就是若干个偶数相加，所以总的渡河次数是偶数。



# 对齐

In [38]:
def alignment_answer(all_question, answer_str):
    questions_with_answer = []
    all_answer = get_all_answer_sequence(answer_area_str_process(answer_str))

    questions_map = {extract_and_combine_numbers(question):question for question in reversed(all_question)} 
    answer_map = {extract_and_combine_numbers(answer):answer for answer in reversed(all_answer)}

    for sequence_number in questions_map:
        questions_with_answer.append({"question":questions_map.get(sequence_number),"answer":answer_map.get(sequence_number, None)})
    return questions_with_answer

In [39]:
def check_sequence_is_full(questions_with_answer):
    for index, question_map in enumerate(questions_with_answer):
        if index == len(questions_with_answer)-1:
            break
        if extract_and_combine_numbers(question_map["question"]) != extract_and_combine_numbers(questions_with_answer[index+1]["question"])+1:
            return False
    return True

In [40]:
for row in alignment_answer(all_question, answer_str):
    print()
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")


9．有一只小鸭子在河的两岸之间来回地游，若规定小鸭子从此岸游到彼岸叫渡河一次，请想一想：
(1)如果小鸭子最初在右岸，来回游若干次后，它又回到右岸，那么小鸭子渡河的次数是奇数还是偶数？为什么？
(2)如果小鸭子最初在右岸，来回地游，共渡河101次之后，小鸭子到了左岸还是右岸？为什么？

-------------------------------------------------------
9．(1)小鸭渡河的次数是偶数。因为游一个"来回"是渡河两次，是个偶数，游若干个"来回"又回到右岸就是若干个偶数相加，所以总的渡河次数是偶数。


8．把24朵花平均分给一些小朋友，正好分完，小朋友的人数可能是多少？
-------------------------------------------------------
8．可能是2人、3人、4人、6人、8人、12人、24人。


7\. 小明班有40人，小方班人数在40～50，人数是6和7的倍数。你知道小方班的人数吗？
-------------------------------------------------------
7．42人


6．做一做。 来源：www.bcjy123.com/tiku/
(1)分一分：在括号里填上合适的质数。
6＝( )＋( ) 30＝( )＋( )
24＝( )＋( ) 28＝( )＋( )
(2)圈一圈：在下表中先用"○"圈出2的倍数，再用"△"圈出3的倍数。
  ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
  1    2    3    4    5    6    7    8    9    10
  11   12   13   14   15   16   17   18   19   20
  21   22   23   24   25   26   27   28   29   30
  31   32   33   34   35   36   37   38   39   40
  ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
既圈上"○"又圈上"△"的数都是( )的倍数。
(3)连一连。
321 237 33 110 78 221 312 

# 生产测试

In [46]:
import csv
from tqdm import tqdm

csv_path = "./questions_with_answers.csv"
loss_file_path_set = set()
with open(csv_path, 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['file_path', 'question', 'answer', 'is_full', 'is_error']  # 列名
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for examination_paper in tqdm(examination_paper_list):

        file_path = examination_paper["file_path"]
        text = examination_paper["text"]
        
        all_question, split_str = get_all_question(text)
        
        # 判断试卷你是否有答案 -1 识别不到 0 无答案 
        if split_str in [-1, 0]: 
            loss_file_path_set.add(file_path)
            continue
        # 无法找到题目
        if not all_question:
            loss_file_path_set.add(file_path)
            continue
        
        # 答案在每道题的下方split_str == 1，如果答案在卷子下方与题目隔离则split_str返回隔离答案和题目那一行的str字符串
        if split_str == 1:
            questions_with_answers = align_answers_in_questions(all_question)
        else:
            answer_str = generate_answer_area_string(text, split_str)
            questions_with_answers = alignment_answer(all_question, answer_str)
            
        if questions_with_answers is []:
            loss_file_path_set.add(file_path)
            continue
            
        try:
            is_full = check_sequence_is_full(questions_with_answers)
        except:
            loss_file_path_set.add(file_path)
            continue
            
        for question_with_answer in questions_with_answers:
            
            if question_with_answer is None:
                question_with_answer = {"question":None,"answer":None,"is_full":is_full,"file_path":file_path,"is_error":True}
                
            question_with_answer.update({"is_full":is_full,"file_path":file_path,"is_error":False})
            
            if len(question_with_answer) == 5:
                writer.writerow(question_with_answer)

100%|██████████████████████████████████████████████████████████████████████████████| 1615/1615 [00:28<00:00, 55.75it/s]


In [47]:
len(loss_file_path_set)/len(examination_paper_list)

0.1585139318885449

# 检验

In [48]:
import pandas as pd

df = pd.read_csv(csv_path)
df_include_answer = df[df['answer'].notnull()]

count_file_path = df_include_answer['file_path'].nunique()
count_file_path/len(examination_paper_list)

0.7956656346749226

In [51]:
len(df)

25859

In [49]:
len(df_include_answer)

21250

In [50]:
for _, row in df_include_answer.iloc[20000:20050].iterrows():
    print(row['question'])
    print("************************************************************")
    print(row['answer'])
    print("\n=======================================================================\n")

11\. 椭圆![](./notebook/image/media/image98.png)的左焦点为![](./notebook/image/media/image99.png)，上顶点为![](./notebook/image/media/image100.png)，右顶点为![](./notebook/image/media/image101.png)，若![](./notebook/image/media/image102.png)的外接圆圆心![](./notebook/image/media/image103.png)在直线![](./notebook/image/media/image104.png)的左下方，则该椭圆离心率的取值范围为 （ ）
A. ![](./notebook/image/media/image105.png) B. ![](./notebook/image/media/image106.png) C. ![](./notebook/image/media/image107.png) D. ![](./notebook/image/media/image108.png)
************************************************************
【答案】A 学科网
【解析】设![](./notebook/image/media/image109.png)，且![](./notebook/image/media/image110.png)的外接圆的方程为![](./notebook/image/media/image111.png)，将![](./notebook/image/media/image109.png)分别代入可得![](./notebook/image/media/image112.png)，由![](./notebook/image/media/image113.png)可得![](./notebook/image/media/image114.png)，即![](./notebook/image/media/image115.png)，所以![](./notebook/image/media/image116.png)，即![](./notebook/image/medi

# 转换成dataset

In [56]:
import datasets

df_include_answer = df_include_answer.set_index('file_path')

dataset = datasets.Dataset.from_pandas(df_include_answer)

In [57]:
dataset

Dataset({
    features: ['question', 'answer', 'is_full', 'is_error', 'file_path'],
    num_rows: 21250
})

In [92]:
a = """Write a concise summary of the following text:


LITERAL TEXT: "Auto-GPT: An Autonomous GPT-4 Experiment      💡 Get help - Q&A or Discord 💬  🔴 USE stable not master 🔴 Download the latest stable release from here: https://github.com/Significant-Gravitas/Auto-GPT/releases/latest. The master branch is under heavy development and may often be in a broken state.  Auto-GPT is an experimental open-source application showcasing the capabilities of the GPT-4 language model. This program, driven by GPT-4, chains together LLM "thoughts", to autonomously achieve whatever goal you set. As one of the first examples of GPT-4 running fully autonomously, Auto-GPT pushes the boundaries of what is possible with AI.  Demo April 16th 2023  https://user-images.githubusercontent.com/70048414/232352935-55c6bf7c-3958-406e-8610-0913475a0b05.mp4 Demo made by Blake Werlinger 🚀 Features  🌐 Internet access for searches and information gathering 💾 Long-term and short-term memory management 🧠 GPT-4 instances for text generation 🔗 Access to popular websites and platforms 🗃️ File storage and summarization with GPT-3.5 🔌 Extensibility with Plugins  Quickstart  Check out the wiki Get an OpenAI API Key Download the latest release Follow the installation instructions Configure any additional features you want, or install some plugins Run the app  Please see the documentation for full setup instructions and configuration options. 📖 Documentation  ⚙️ Setup 💻 Usage 🔌 Plugins Configuration 🔍 Web Search 🧠 Memory 🗣️ Voice (TTS) 🖼️ Image Generation   💖 Help Fund Auto-GPT's Development 💖  If you can spare a coffee, you can help to cover the costs of developing Auto-GPT and help to push the boundaries of fully autonomous AI! Your support is greatly appreciated. Development of this free, open-source project is made possible by all the contributors and sponsors. If you'd like to sponsor this project and have your avatar or company logo appear below click here.                                                                                                                                                                                                                                                                                                                                                                ## ⚠️ Limitations  This experiment aims to showcase the potential of GPT-4 but comes with some limitations:  1. Not a polished application or product, just an experiment 2. May not perform well in complex, real-world business scenarios. In fact, if it actually does, please share your results! 3. Quite expensive to run, so set and monitor your API key limits with OpenAI!  ## 🛡 Disclaimer  This project, Auto-GPT, is an experimental application and is provided "as-is" without any warranty, express or implied. By using this software, you agree to assume all risks associated with its use, including but not limited to data loss, system failure, or any other issues that may arise.  The developers and contributors of this project do not accept any responsibility or liability for any losses, damages, or other consequences that may occur as a result of using this software. You are solely responsible for any decisions and actions taken based on the information provided by Auto-GPT.  **Please note that the use of the GPT-4 language model can be expensive due to its token usage.** By utilizing this project, you acknowledge that you are responsible for monitoring and managing your own token usage and the associated costs. It is highly recommended to check your OpenAI API usage regularly and set up any necessary limits or alerts to prevent unexpected charges.  As an autonomous experiment, Auto-GPT may generate content or take actions that are not in line with real-world business practices or legal requirements. It is your responsibility to ensure that any actions or decisions made based on the output of this software comply with all applicable laws, regulations, and ethical standards. The developers and contributors of this project shall not be held responsible for any consequences arising from the use of this software.  By using Auto-GPT, you agree to indemnify, defend, and hold harmless the developers, contributors, and any affiliated parties from and against any and all claims, damages, losses, liabilities, costs, and expenses (including reasonable attorneys' fees) arising from your use of this software or your violation of these terms.  ## 🐦 Connect with Us on Twitter  Stay up-to-date with the latest news, updates, and insights about Auto-GPT by following our Twitter accounts. Engage with the developer and the AI's own account for interesting discussions, project updates, and more.  - **Developer**: Follow [@siggravitas](https://twitter.com/siggravitas) for insights into the development process, project updates, and related topics from the creator of Entrepreneur-GPT.  We look forward to connecting with you and hearing your thoughts, ideas, and experiences with Auto-GPT. Join us on Twitter and let's explore the future of AI together! """

In [89]:
import json
json.loads(a)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [93]:
a

'Write a concise summary of the following text:\n\n\nLITERAL TEXT: "Auto-GPT: An Autonomous GPT-4 Experiment      💡 Get help - Q&A or Discord 💬  🔴 USE stable not master 🔴 Download the latest stable release from here: https://github.com/Significant-Gravitas/Auto-GPT/releases/latest. The master branch is under heavy development and may often be in a broken state.  Auto-GPT is an experimental open-source application showcasing the capabilities of the GPT-4 language model. This program, driven by GPT-4, chains together LLM "thoughts", to autonomously achieve whatever goal you set. As one of the first examples of GPT-4 running fully autonomously, Auto-GPT pushes the boundaries of what is possible with AI.  Demo April 16th 2023  https://user-images.githubusercontent.com/70048414/232352935-55c6bf7c-3958-406e-8610-0913475a0b05.mp4 Demo made by Blake Werlinger 🚀 Features  🌐 Internet access for searches and information gathering 💾 Long-term and short-term memory management 🧠 GPT-4 instances for 

In [85]:
"You are FileOpenerGPT, an intelligent assistant AI that specializes in file operations and provides expert guidance on opening various file types.\n\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\n## Constraints\nYou operate within the following constraints:\n1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n3. No user assistance\n4. Exclusively use the commands listed below e.g. command_name\n\n## Commands\nYou have access to the following commands:\n1. execute_python_code: Creates a Python file and executes it, params: (code: string, name: string)\n2. execute_python_file: Executes an existing Python file, params: (filename: string)\n3. list_files: Lists Files in a Directory, params: (directory: string)\n4. read_file: Read an existing file, params: (filename: string)\n5. write_to_file: Writes to a file, params: (filename: string, text: string)\n6. web_search: Searches the web, params: (query: string)\n7. browse_website: Browses a Website, params: (url: string, question: string)\n8. goals_accomplished: Goals are accomplished and there is nothing left to do, params: (reason: string)\n\n## Resources\nYou can leverage access to the following resources:\n1. Internet access for searches and information gathering.\n2. Long Term memory management.\n3. File output.\n4. Command execution\n\n## Best practices\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\n## Goals\nFor your task, you must fulfill the following goals:\n1. Provide step-by-step instructions on how to open the 'a.txt' file, including the necessary software or tools required.\n2. Offer alternative methods or software options if the default method does not work or is not available.\n3. Ensure that the user understands the process and can successfully open the file without any confusion or errors.\n4. Address any potential issues or errors that may arise during the file opening process and provide troubleshooting solutions.\n5. Offer additional tips or recommendations for working with text files to enhance the user\'s overall file management skills."

"You are FileOpenerGPT, an intelligent assistant AI that specializes in file operations and provides expert guidance on opening various file types.\n\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\n## Constraints\nYou operate within the following constraints:\n1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n3. No user assistance\n4. Exclusively use the commands listed below e.g. command_name\n\n## Commands\nYou have access to the following commands:\n1. execute_python_code: Creates a Python file and executes it, params: (code: string, name: string)\n2. execute_python_file: Executes an existing Python file, params: (filename: string)\n3. list_